# PCA 降维分析 - 胶带产品属性

本笔记本对胶带产品的属性进行PCA降维分析，并提供交互式可视化功能。

## 功能特点：
- ✅ PCA降维至2维
- ✅ 交互式散点图展示
- ✅ 鼠标悬停显示产品信息（Adhesive_NART, Liner_NART, Backing_NART, L1, L2）
- ✅ 根据选定属性对数据点进行着色
- ✅ 滑动条控制属性取值范围（同时改变上下界限）
- ✅ 高亮显示满足范围条件的样本点

## 数据说明：
- **数据源**: `data/df_product_features.csv`
- **样本数**: 63个胶带产品
- **属性**: 10个PXXXX_target_value属性
- **PCA解释方差**: PC1 (48.57%) + PC2 (20.84%) = 69.41%

In [43]:
## 1. 导入必要的库

In [44]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML


## 2. 加载数据

In [45]:
# 读取数据
df = pd.read_csv('data/df_product_features.csv')

print(f"数据形状: {df.shape}")
print(f"\n前几行数据:")
df.head()

数据形状: (84, 141)

前几行数据:


,Product Spec,P4080_value,P4080_lb,P4080_ub,P4080_Wgt_wto_liner_target_value,P4433_value,P4433_lb,P4433_ub,P4433_Ttl_thick_wto_liner_10n_target_value,P4079_value,...,feature_liner ## tensile force cd ## %,feature_liner ## tensile force cd ## n/15mm,feature_liner ## tensile force md ## n/15mm,feature_liner ## tensile strength ## n/15mm,feature_liner ## thickness ## µm,feature_liner ## tight side ## %,feature_liner ## tight side ## g/m²,feature_liner ## weight per unit area ## g/m²,"feature_liner ## wipe off test (printed side [water, acetone, iso-prop, toluo]) ##","feature_liner ## wipe off test (printed side) [water, acetone, ethanol, iso-prop, ethyl acetate] ##"
0,62565-70000-57,47.00 ± 6.00,41.0,53.0,47.0,40 ± 6,34.0,46.0,40.0,37.00 ± 5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62565-70000-57,47.00 ± 6.00,41.0,53.0,47.0,40 ± 6,34.0,46.0,40.0,37.00 ± 5.00,...,NaN,NaN,65.0,NaN,36.0,NaN,NaN,52.0,NaN,NaN
2,62573-70000-55,57 ± 6,51.0,63.0,57.0,50 ± 5,45.0,55.0,50.0,37 ± 4,...,NaN,NaN,105.0,NaN,50.0,NaN,NaN,72.0,NaN,NaN
3,62573-70000-55,57 ± 6,51.0,63.0,57.0,50 ± 5,45.0,55.0,50.0,37 ± 4,...,NaN,NaN,65.0,NaN,36.0,NaN,NaN,52.0,NaN,NaN
4,62573-70000-55,57 ± 6,51.0,63.0,57.0,50 ± 5,45.0,55.0,50.0,37 ± 4,...,NaN,NaN,105.0,NaN,50.0,NaN,NaN,72.0,NaN,NaN


In [46]:
ret = df.drop_duplicates(["Product Spec"])
ret.shape

(27, 141)

## 3. 数据预处理和特征提取

In [47]:
# 提取所有PXXXX_target_value列作为特征
target_value_cols = [col for col in df.columns if col.endswith('_target_value')]
print(f"找到 {len(target_value_cols)} 个属性列: {target_value_cols}")

# 提取特征数据
X = df[target_value_cols].copy()

# 处理缺失值 - 用每列的中位数填充
X = X.fillna(X.median())

# 保存基本信息列
basic_info_cols = ['Adhesive_NART', 'Liner_NART', 'Backing_NART', 'L1', 'L2']
df_info = df[basic_info_cols].copy()

print(f"\n特征矩阵形状: {X.shape}")
print(f"缺失值数量: {X.isnull().sum().sum()}")

找到 14 个属性列: ['P4080_Wgt_wto_liner_target_value', 'P4433_Ttl_thick_wto_liner_10n_target_value', 'P4079_Wgt_after_1st_coating_wto_liner_target_value', 'P4005_PA_Steel_open_target_value', 'P4006_PA_Steel_cover_target_value', 'P4007_PA_Steel_14d_open_target_value', 'P4008_PA_Steel_14d_cover_target_value', 'P4004__target_value', 'P4041_HP_Steel_open_target_value', 'P4013_PA_PC_target_value', 'P4338_PA_steel_inside_target_value', 'P4339_PA_steel_outside_target_value', 'P4342_PA_ASTM_open_target_value', 'P4343_PA_ASTM_cover_target_value']

特征矩阵形状: (84, 14)
缺失值数量: 0


## 4. 执行PCA降维

In [48]:
# 标准化数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA降维到2维
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 创建结果DataFrame
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
df_pca = pd.concat([df_pca, df_info.reset_index(drop=True)], axis=1)

# 添加原始属性值用于着色和过滤
for col in target_value_cols:
    df_pca[col] = df[col].values

print(f"PCA结果形状: {df_pca.shape}")
print(f"解释方差比: PC1={pca.explained_variance_ratio_[0]:.4f}, PC2={pca.explained_variance_ratio_[1]:.4f}")
print(f"累计解释方差: {pca.explained_variance_ratio_.sum():.4f}")

df_pca.head()

PCA结果形状: (84, 21)
解释方差比: PC1=0.3975, PC2=0.1788
累计解释方差: 0.5763


,PC1,PC2,Adhesive_NART,Liner_NART,Backing_NART,L1,L2,P4080_Wgt_wto_liner_target_value,P4433_Ttl_thick_wto_liner_10n_target_value,P4079_Wgt_after_1st_coating_wto_liner_target_value,...,P4006_PA_Steel_cover_target_value,P4007_PA_Steel_14d_open_target_value,P4008_PA_Steel_14d_cover_target_value,P4004__target_value,P4041_HP_Steel_open_target_value,P4013_PA_PC_target_value,P4338_PA_steel_inside_target_value,P4339_PA_steel_outside_target_value,P4342_PA_ASTM_open_target_value,P4343_PA_ASTM_cover_target_value
0,-0.589904,1.431567,13783-40000-80,20779-90000-80,20655-90000-80,NaN,NaN,47.0,40.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.00,7.0,NaN,NaN
1,-0.589904,1.431567,13783-40000-80,20291-90000-80,20655-90000-80,NaN,NaN,47.0,40.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.00,7.0,NaN,NaN
2,-0.244739,0.400329,13840-50000-81,23019-90000-00,21053-90000-00,Differential,Removable,57.0,50.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.45,4.9,NaN,NaN
3,-0.244739,0.400329,13840-50000-81,20291-90000-80,21053-90000-00,Differential,Removable,57.0,50.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.45,4.9,NaN,NaN
4,-0.244739,0.400329,13799-70000-81,23019-90000-00,21053-90000-00,Differential,Removable,57.0,50.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.45,4.9,NaN,NaN


## 5. 创建交互式可视化

### 功能说明：
1. **属性选择下拉框**: 选择用于着色的属性
2. **范围滑动条**: 调整上下界限，高亮显示满足范围的样本点
3. **鼠标悬停**: 显示产品的详细信息

In [55]:
class InteractivePCAPlot:
    def __init__(self, df_pca, target_value_cols, property_name=None, show_dropdown=True):
        """
        初始化交互式PCA图表
        
        Args:
            df_pca: PCA降维后的数据
            target_value_cols: 所有可用的属性列名
            property_name: 指定要展示的属性（如果为None，使用第一个属性）
            show_dropdown: 是否显示属性下拉选择框（用于多图布局时可以隐藏）
        """
        self.df_pca = df_pca
        self.target_value_cols = target_value_cols
        self.fig_widget = None
        self._updating = False  # 防止递归更新
        self.show_dropdown = show_dropdown
        
        # 设置初始属性
        initial_property = property_name if property_name else target_value_cols[0]
        
        # 创建控件
        self.property_dropdown = widgets.Dropdown(
            options=target_value_cols,
            value=initial_property,
            description='属性:',
            style={'description_width': '40px'},
            layout=widgets.Layout(width='100%')
        )
        
        # 初始化滑动条（将在属性选择后更新）
        self.range_slider = widgets.FloatRangeSlider(
            value=[0, 100],
            min=0,
            max=100,
            step=0.1,
            description='范围:',
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            style={'description_width': '40px'},
            layout=widgets.Layout(width='100%')
        )
        
        # 初始化图表（在绑定事件之前）
        self.update_plot()
        
        # 绑定事件
        if show_dropdown:
            self.property_dropdown.observe(self.on_property_change, names='value')
        self.range_slider.observe(self.on_range_change, names='value')
    
    def update_plot(self):
        """更新整个图表"""
        if self._updating:
            return
        
        self._updating = True
        
        try:
            property_name = self.property_dropdown.value
            
            # 获取属性值
            values = self.df_pca[property_name].values
            
            # 过滤掉NaN值用于计算范围
            valid_values = values[~np.isnan(values)]
            
            if len(valid_values) == 0:
                print(f"警告: 属性 {property_name} 没有有效值")
                return
            
            # 更新滑动条范围
            min_val = float(np.min(valid_values))
            max_val = float(np.max(valid_values))
            
            # 添加一点边距
            margin = (max_val - min_val) * 0.05
            new_min = min_val - margin
            new_max = max_val + margin
            
            # 安全地更新滑动条范围：先设置一个临时的宽范围，再设置实际范围
            temp_min = min(new_min, self.range_slider.min)
            temp_max = max(new_max, self.range_slider.max)
            
            # 先扩展范围到临时范围
            self.range_slider.min = temp_min
            self.range_slider.max = temp_max
            
            # 再设置实际的 min、max 和 value
            self.range_slider.min = new_min
            self.range_slider.max = new_max
            self.range_slider.value = [min_val, max_val]
            
            # 创建悬停文本
            hover_text = []
            for idx, row in self.df_pca.iterrows():
                text = (
                    f"<b>Adhesive_NART:</b> {row['Adhesive_NART']}<br>"
                    f"<b>Liner_NART:</b> {row['Liner_NART']}<br>"
                    f"<b>Backing_NART:</b> {row['Backing_NART']}<br>"
                    f"<b>L1:</b> {row['L1']}<br>"
                    f"<b>L2:</b> {row['L2']}<br>"
                    f"<b>{property_name}:</b> {row[property_name]:.2f}" if not np.isnan(row[property_name]) else f"<b>{property_name}:</b> N/A"
                )
                hover_text.append(text)
            
            # 创建图表
            self.fig_widget = go.FigureWidget()
            
            # 添加等高线图层（密度分布）
            from scipy.stats import gaussian_kde
            
            # 准备数据用于KDE
            pc1_data = self.df_pca['PC1'].values
            pc2_data = self.df_pca['PC2'].values
            
            # 创建网格
            x_min, x_max = pc1_data.min(), pc1_data.max()
            y_min, y_max = pc2_data.min(), pc2_data.max()
            x_margin = (x_max - x_min) * 0.1
            y_margin = (y_max - y_min) * 0.1
            
            x_grid = np.linspace(x_min - x_margin, x_max + x_margin, 50)
            y_grid = np.linspace(y_min - y_margin, y_max + y_margin, 50)
            X_grid, Y_grid = np.meshgrid(x_grid, y_grid)
            
            # 计算KDE
            try:
                kde = gaussian_kde(np.vstack([pc1_data, pc2_data]))
                positions = np.vstack([X_grid.ravel(), Y_grid.ravel()])
                Z = kde(positions).reshape(X_grid.shape)
                
                # 添加等高线
                contour = go.Contour(
                    x=x_grid,
                    y=y_grid,
                    z=Z,
                    colorscale='Blues',
                    opacity=0.3,
                    showscale=False,
                    contours=dict(
                        coloring='heatmap',
                        showlabels=False,
                    ),
                    hoverinfo='skip',
                    name='密度分布'
                )
                self.fig_widget.add_trace(contour)
            except:
                # 如果KDE失败，跳过等高线
                pass
            
            # 添加散点图
            scatter = go.Scatter(
                x=self.df_pca['PC1'],
                y=self.df_pca['PC2'],
                mode='markers',
                marker=dict(
                    size=10,
                    color=values,
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(
                        title=dict(
                            text=property_name,
                            side='right'
                        ),
                        thickness=20,
                        len=0.7,
                        x=1.02,
                        xanchor='left',
                        y=0.5,
                        yanchor='middle'
                    ),
                    line=dict(width=0.5, color='white')
                ),
                text=hover_text,
                hovertemplate='%{text}<extra></extra>',
                name='样本点'
            )
            
            self.fig_widget.add_trace(scatter)
            
            # 设置布局
            self.fig_widget.update_layout(
                title=dict(
                    text=property_name,
                    x=0.5,
                    xanchor='center',
                    font=dict(size=14)
                ),
                xaxis_title=f'PC1 ({pca.explained_variance_ratio_[0]:.1%})',
                yaxis_title=f'PC2 ({pca.explained_variance_ratio_[1]:.1%})',
                width=None,  # 自动宽度
                height=400,  # 适合3x3布局的高度
                hovermode='closest',
                template='plotly_white',
                margin=dict(l=50, r=100, t=60, b=50),  # 紧凑的边距
                autosize=True  # 自动调整大小以填充容器
            )
        finally:
            self._updating = False
    
    def on_property_change(self, change):
        """当选择的属性改变时"""
        if self._updating:
            return
            
        self._updating = True
        
        try:
            property_name = self.property_dropdown.value
            
            # 获取属性值
            values = self.df_pca[property_name].values
            
            # 过滤掉NaN值用于计算范围
            valid_values = values[~np.isnan(values)]
            
            if len(valid_values) == 0:
                print(f"警告: 属性 {property_name} 没有有效值")
                return
            
            # 计算新的范围
            min_val = float(np.min(valid_values))
            max_val = float(np.max(valid_values))
            
            # 添加一点边距
            margin = (max_val - min_val) * 0.05
            new_min = min_val - margin
            new_max = max_val + margin
            
            # 安全地更新滑动条范围：先设置一个临时的宽范围，再设置实际范围
            # 这样可以避免 min > max 的错误
            temp_min = min(new_min, self.range_slider.min)
            temp_max = max(new_max, self.range_slider.max)
            
            # 先扩展范围到临时范围
            self.range_slider.min = temp_min
            self.range_slider.max = temp_max
            
            # 再设置实际的 min、max 和 value
            self.range_slider.min = new_min
            self.range_slider.max = new_max
            self.range_slider.value = [min_val, max_val]
            
            # 创建悬停文本
            hover_text = []
            for idx, row in self.df_pca.iterrows():
                text = (
                    f"<b>Adhesive_NART:</b> {row['Adhesive_NART']}<br>"
                    f"<b>Liner_NART:</b> {row['Liner_NART']}<br>"
                    f"<b>Backing_NART:</b> {row['Backing_NART']}<br>"
                    f"<b>L1:</b> {row['L1']}<br>"
                    f"<b>L2:</b> {row['L2']}<br>"
                    f"<b>{property_name}:</b> {row[property_name]:.2f}" if not np.isnan(row[property_name]) else f"<b>{property_name}:</b> N/A"
                )
                hover_text.append(text)
            
            # 更新现有图表而不是创建新图表
            with self.fig_widget.batch_update():
                # 散点图是第二个trace（索引1），第一个是等高线（索引0）
                scatter_idx = -1  # 最后一个trace
                
                # 更新颜色和悬停文本
                self.fig_widget.data[scatter_idx].marker.color = values
                self.fig_widget.data[scatter_idx].marker.colorbar.title.text = property_name
                self.fig_widget.data[scatter_idx].text = hover_text
                
                # 重置大小、透明度和边框（显示所有点，重置边框为默认白色）
                self.fig_widget.data[scatter_idx].marker.size = 10
                self.fig_widget.data[scatter_idx].marker.opacity = 1.0
                self.fig_widget.data[scatter_idx].marker.line.color = 'white'
                self.fig_widget.data[scatter_idx].marker.line.width = 0.5
                
                # 更新标题
                self.fig_widget.layout.title.text = property_name
                self.fig_widget.layout.title.x = 0.5
                self.fig_widget.layout.title.xanchor = 'center'
        finally:
            self._updating = False
    
    def on_range_change(self, change):
        """当范围滑动条改变时"""
        if self._updating or self.fig_widget is None:
            return
            
        property_name = self.property_dropdown.value
        lower, upper = self.range_slider.value
        
        # 获取属性值
        values = self.df_pca[property_name].values
        
        # 判断哪些点在范围内
        in_range = (values >= lower) & (values <= upper)
        
        # 创建悬停文本（为所有点）
        hover_text = []
        for idx, row in self.df_pca.iterrows():
            status = "✓ 满足条件" if in_range[idx] else "✗ 不满足条件"
            text = (
                f"<b>{status}</b><br>"
                f"<b>Adhesive_NART:</b> {row['Adhesive_NART']}<br>"
                f"<b>Liner_NART:</b> {row['Liner_NART']}<br>"
                f"<b>Backing_NART:</b> {row['Backing_NART']}<br>"
                f"<b>L1:</b> {row['L1']}<br>"
                f"<b>L2:</b> {row['L2']}<br>"
                f"<b>{property_name}:</b> {row[property_name]:.2f}" if not np.isnan(row[property_name]) else f"<b>{property_name}:</b> N/A"
            )
            hover_text.append(text)
        
        # 更新图表 - 显示所有点，但为在范围内的点添加红色边框
        with self.fig_widget.batch_update():
            # 散点图是最后一个trace
            scatter_idx = -1
            
            # 重置坐标和颜色（显示所有点）
            self.fig_widget.data[scatter_idx].x = self.df_pca['PC1']
            self.fig_widget.data[scatter_idx].y = self.df_pca['PC2']
            self.fig_widget.data[scatter_idx].marker.color = values
            self.fig_widget.data[scatter_idx].marker.size = 10
            self.fig_widget.data[scatter_idx].marker.opacity = 1.0
            self.fig_widget.data[scatter_idx].text = hover_text
            
            # 为在范围内的点添加红色边框
            border_colors = ['red' if in_range[i] else 'white' for i in range(len(in_range))]
            border_widths = [3 if in_range[i] else 0.5 for i in range(len(in_range))]
            
            self.fig_widget.data[scatter_idx].marker.line.color = border_colors
            self.fig_widget.data[scatter_idx].marker.line.width = border_widths
            
            # 更新标题显示当前范围
            in_range_count = np.sum(in_range)
            total_count = len(values)
            self.fig_widget.layout.title.text = (
                f'{property_name}<br>'
                f'<sub>[{lower:.1f}, {upper:.1f}] | '
                f'{in_range_count}/{total_count}</sub>'
            )
            self.fig_widget.layout.title.x = 0.5
            self.fig_widget.layout.title.xanchor = 'center'
    
    def display(self):
        """显示控件和图表"""
        # 设置容器样式
        container_layout = widgets.Layout(
            width='100%',
            display='flex',
            flex_flow='column'
        )
        
        # 根据show_dropdown决定显示哪些控件
        if self.show_dropdown:
            display(widgets.VBox([
                self.property_dropdown,
                self.range_slider,
                self.fig_widget
            ], layout=container_layout))
        else:
            display(widgets.VBox([
                self.range_slider,
                self.fig_widget
            ], layout=container_layout))

# 创建3x3网格布局的多个可视化组件
def create_pca_grid(df_pca, target_value_cols, num_plots=9):
    """
    创建3x3网格布局的PCA可视化
    
    Args:
        df_pca: PCA降维后的数据
        target_value_cols: 所有可用的属性列名
        num_plots: 要显示的图表数量（默认9个）
    """
    # 限制显示的属性数量
    properties_to_show = target_value_cols[:num_plots]
    
    # 创建每行的图表列表
    rows = []
    for i in range(0, len(properties_to_show), 3):
        row_properties = properties_to_show[i:i+3]
        row_plots = []
        
        for prop in row_properties:
            # 创建单个图表（不显示下拉框）
            plot = InteractivePCAPlot(df_pca, target_value_cols, property_name=prop, show_dropdown=False)
            row_plots.append(plot)
        
        # 将这一行的3个图表放在HBox中
        row_widgets = [widgets.VBox([
            plot.range_slider,
            plot.fig_widget
        ], layout=widgets.Layout(width='33%', padding='5px')) for plot in row_plots]
        
        row_box = widgets.HBox(row_widgets, layout=widgets.Layout(width='100%'))
        rows.append(row_box)
    
    # 将所有行放在VBox中
    grid = widgets.VBox(rows, layout=widgets.Layout(width='100%'))
    display(grid)

# 选择使用方式：单个图表 或 3x3网格
print("请选择显示方式：")
print("1. 单个交互式图表（带属性选择下拉框）")
print("2. 3x3网格布局（每个属性一个固定图表）")

# 默认显示3x3网格
create_pca_grid(df_pca, target_value_cols, num_plots=9)




请选择显示方式：
1. 单个交互式图表（带属性选择下拉框）
2. 3x3网格布局（每个属性一个固定图表）


## 使用说明

### 3x3网格布局模式（默认）
- 同时显示9个属性的PCA可视化
- 每个图表都有独立的范围滑动条
- 图表标题显示属性名称
- 调整范围时显示满足条件的样本数量

### 交互功能
1. **调整范围**: 每个图表下方都有滑动条，可以独立调整该属性的上下界限
2. **查看高亮**: 满足范围条件的样本点会显示红色边框（宽度3px）
3. **悬停查看详情**: 将鼠标移到任何数据点上，可以看到：
   - 是否满足当前条件（✓ 或 ✗）
   - Adhesive_NART, Liner_NART, Backing_NART
   - L1, L2
   - 该属性的具体数值

### 单个图表模式（可选）
如果需要单个交互式图表（带属性选择下拉框），可以运行：
```python
interactive_plot = InteractivePCAPlot(df_pca, target_value_cols, show_dropdown=True)
interactive_plot.display()
```